In [19]:
import torch
#import torchvision
#import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import KddModel as kdd

In [20]:
batch_size=100
root='../../data/kdd/'
transform=torch.FloatTensor

#dataset
trainset=kdd.Kddset("KDDTrain+.txt",train=True,transform=transform)
testset=kdd.Kddset("KDDTest+.txt",train=False,transform=transform)

#dataloaders
trainloader=torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True)
testloader=torch.utils.data.DataLoader(testset,batch_size=batch_size)

#constant for classes
classes=('normaly','anormal')

In [21]:
#CNN
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.convmodel=nn.Sequential(nn.Conv2d(1,6,3,padding=2),
                                    nn.BatchNorm2d(6),
                                    nn.Sigmoid(),
                                    nn.Conv2d(6,16,3),
                                    nn.BatchNorm2d(16),
                                    nn.Sigmoid(),
                                    )
        self.fcmodel=nn.Sequential(nn.Linear(16*7*7,200),
                                    nn.BatchNorm1d(200),
                                    nn.Sigmoid(),
                                    nn.Linear(200,20),
                                    nn.Sigmoid(),
                                    nn.Linear(20,2)
                                    )

    def forward(self,out):
        out=self.convmodel(out)
        out=out.view(-1,16*7*7)
        out=self.fcmodel(out)
        return out


In [22]:
def accuracy(net,testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            cvdata, labels = data
            outputs = net(cvdata)
            _, predicted = torch.max(outputs.data, 1)
            predicted[predicted>0]=1.
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    #print("correct:%d  total:%d" %(correct,total))
    print('Accuracy: %d %%' % (100 * correct / total))

In [23]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-2)
#scheduler = optim.lr_scheduler.StepLR(optimizer, 6,gamma=0.1)

In [24]:
running_loss = 0.0
mini_batches_count=200
for epoch in range(4):  # loop over the dataset multiple times
    for i, data in enumerate(trainloader, 0):

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # whatur(labels)
        # break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # every 1000 mini-batches...
        if i % mini_batches_count == (mini_batches_count-1):    
            #scheduler.step()
            print("[%d,%6d] ava_loss: %.5f"%(epoch+1,i+1,running_loss/mini_batches_count))    
            running_loss = 0.0

    accuracy(net,testloader)
print('Finished Training')

[1,   200] ava_loss: 0.25556
[1,   400] ava_loss: 0.14957
[1,   600] ava_loss: 0.13247
[1,   800] ava_loss: 0.09738
[1,  1000] ava_loss: 0.09551
[1,  1200] ava_loss: 0.08646
Accuracy: 77 %
[2,   200] ava_loss: 0.10152
[2,   400] ava_loss: 0.08627
[2,   600] ava_loss: 0.05362
[2,   800] ava_loss: 0.05430
[2,  1000] ava_loss: 0.07013
[2,  1200] ava_loss: 0.06109
Accuracy: 78 %
[3,   200] ava_loss: 0.06241
[3,   400] ava_loss: 0.04467
[3,   600] ava_loss: 0.04927
[3,   800] ava_loss: 0.03949
[3,  1000] ava_loss: 0.04580
[3,  1200] ava_loss: 0.04348
Accuracy: 81 %
[4,   200] ava_loss: 0.05800
[4,   400] ava_loss: 0.03959
[4,   600] ava_loss: 0.03232
[4,   800] ava_loss: 0.02916
[4,  1000] ava_loss: 0.03384
[4,  1200] ava_loss: 0.03019
Accuracy: 81 %
Finished Training


In [25]:
PATH="../../pth/nsl_kdd_test.pth"
torch.save(net.state_dict(),PATH)

In [26]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [27]:
class_correct = list(0 for i in range(2))
class_total = list(0 for i in range(2))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        predicted[predicted>0]=1
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(2):
    print('total:%d,correct:%d' %(class_total[i],class_correct[i]))
    print('Accuracy: %.4f %%' % (100 * class_correct[i] / class_total[i]))

total:9711,correct:9408
Accuracy: 96.8798 %
total:12833,correct:8992
Accuracy: 70.0694 %
